In [1]:
using Luna
using  DelimitedFiles 
import PyPlot:pygui, plt
close("all")
pygui(true)

in_dir="input"
L = 3e-3

p_ed = 1e-3
pres= 1.0 
gas = :Ar


file_ρ    = "dens_$(pres)bar.dat"  # name of density profile data file 
path_ρ    = joinpath(in_dir, file_ρ)                  # sys. path to density profile data file 

z_in = readdlm(path_ρ,' ', Float64, '\n')[:,1]        # read in spatial points 
ρ_in = readdlm(path_ρ,' ', Float64, '\n')[:,2]        # read in density data 

dens_coms = Maths.CSpline(z_in, ρ_in)                      # interpolate density function 
    
        
    
Z= (0, L/2, L)                                        # define points for pressure gradient
P= (p_ed, pres, p_ed)  # define values for pressure gradient (see "p_const" above)
(coren,dens_grad)=Capillary.gradient(gas,Z,P)              # gives n(ω; z) and ρ(z) from pressure profile   
       

UndefVarError: UndefVarError: `gas` not defined

In [ ]:
dens_coms()